In [1]:
import pandas as pd

In [2]:
%%time

train = pd.read_parquet('data/data_fusion_train.parquet')

CPU times: user 13.1 s, sys: 4.69 s, total: 17.8 s
Wall time: 16.4 s


In [3]:
%%time

train = train[train.category_id != -1]
train['weight'] = 1
train_unique = train.groupby('item_name').agg({'category_id': 'first', 'weight': 'sum'}).reset_index()

CPU times: user 2.09 s, sys: 1.36 s, total: 3.45 s
Wall time: 2.94 s


In [4]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000)
X_train = tfidf.fit_transform(train_unique.item_name)
y_train = train_unique['category_id']

CPU times: user 3.21 s, sys: 5.46 s, total: 8.66 s
Wall time: 893 ms


In [5]:
class_weights = train_unique.groupby('category_id').agg({'weight': 'sum'})
class_weights /= class_weights.sum()

class_weights = class_weights.sort_values('weight', ascending=False)
class_weights

,weight
category_id,
203,0.205393
84,0.138665
80,0.102909
78,0.077039
71,0.065114
...,...
100,0.000048
1,0.000035
106,0.000028


In [6]:
class_weights = class_weights.to_dict()['weight']

In [7]:
from sklearn.metrics import f1_score, make_scorer
import numpy as np

def f1_weighted(y, p):
    resulted_f1 = []
    for c in y.unique():
        f1 = f1_score(y == c, p == c)
        resulted_f1.append(f1 * class_weights[c])
        
    return np.sum(resulted_f1) 

f1_weighted_sc = make_scorer(f1_weighted)

In [8]:
%%time

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

clf = LinearSVC()

N_JOBS=8
scores = cross_val_score(clf, X_train, y_train, cv=N_JOBS, scoring=f1_weighted_sc, n_jobs=N_JOBS)

CPU times: user 116 ms, sys: 403 ms, total: 518 ms
Wall time: 5.13 s


In [9]:
np.mean(scores), np.std(scores)

(0.7216731030879311, 0.06171714476070219)

In [10]:
%%time

_ = clf.fit(X_train, y_train)

CPU times: user 3.1 s, sys: 27.1 ms, total: 3.13 s
Wall time: 3.13 s


In [11]:
import pickle

pickle.dump(tfidf, open('t1_sub/tfidf', 'wb'))
pickle.dump(clf, open('t1_sub/clf_task1', 'wb'))

In [12]:
!ls -lh t1_sub

итого 12M
-rw-r--r-- 1 dmitry.dremov dmitry.dremov  312 янв 30 21:37 answers.csv
-rw-r--r-- 1 dmitry.dremov dmitry.dremov 7,4M янв 30 21:38 clf_task1
drwxr-xr-x 2 dmitry.dremov dmitry.dremov 4,0K янв 30 18:06 data
-rw-r--r-- 1 dmitry.dremov dmitry.dremov  352 янв 30 18:11 script.py
-rw-r--r-- 1 dmitry.dremov dmitry.dremov 2,9M янв 30 21:38 submission.zip
-rw-r--r-- 1 dmitry.dremov dmitry.dremov 1,3M янв 30 21:38 tfidf


In [13]:
import zipfile 
compression = zipfile.ZIP_DEFLATED

submission_name = 't1_sub/submission.zip'
with zipfile.ZipFile(submission_name, 'w') as zipObj:
    for filename in [
        'clf_task1',
        'tfidf',
        'script.py',
    ]:
        zipObj.write(
            f't1_sub/{filename}', 
            arcname=filename, 
            compress_type=compression
        )
    print(zipObj.namelist())

!ls -lh {submission_name}

['clf_task1', 'tfidf', 'script.py']
-rw-r--r-- 1 dmitry.dremov dmitry.dremov 2,9M янв 30 21:38 t1_sub/submission.zip


# Debug run

In [14]:
!cd t1_sub; python3 script.py

In [15]:
pd.read_csv('t1_sub/answers.csv').head(5)

,id,pred
0,0,80
1,1,204
2,2,204
3,3,71
4,4,79
